## Bridging Biological and Artificial Intelligence

This notebook explores how biological neurons (Hodgkin-Huxley model) relate to artificial neurons in deep learning. Understanding these parallels lies at the heart of **NeuroAI**, using neuroscience to inspire AI, and AI to understand the brain.

**Learning Goals**
- Understand differences between biological and artificial neurons
- Explore insights from neuroscience that inform AI
- Relate the Hodgkin-Huxley model to modern architectures
- Connect parameter inference in biology and AI

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys

sys.path.append("pyHH/src")
from pyhh import HHModel, Simulation

%matplotlib inline

### Simulating a Biological Neuron

Generate a typical action potential using the Hodgkin-Huxley model (biologically grounded description of a neuron’s response to input current) using `pyhh`.

In [2]:
def simulate_hh_spike(V_rest=50.0, stim_amplitude=5.0):
    """Simulate HH model and return membrane potential trace."""
    model = HHModel()
    model.V_rest = V_rest
    
    stim = np.zeros(20000)
    stim[7000:13000] = stim_amplitude  # 60 ms pulse
    
    sim = Simulation(model)
    sim.Run(stimulusWaveform=stim, stepSizeMs=0.01)
    return sim.Vm

hh_voltage = simulate_hh_spike()
time_hh = np.arange(len(hh_voltage)) * 0.01

print(f"Simulated {len(hh_voltage)} points ({time_hh[-1]:.1f} ms)")
print(f"Peak voltage: {np.max(hh_voltage):.2f} mV")
print(f"Resting voltage: {hh_voltage[0]:.2f} mV")

simulating 20000 time points...
simulation complete
Simulated 20000 points (200.0 ms)
Peak voltage: 104.32 mV
Resting voltage: -0.01 mV


### Artificial Neuron Activation Functions

Artificial neurons use simple nonlinear functions (ReLU, sigmoid, tanh) to introduce complexity. Let’s define and visualize them.

In [3]:
x = np.linspace(-80, 40, 1000)

def relu(x): return np.maximum(0, x + 65)
def sigmoid(x): return 100 / (1 + np.exp(-x/10))
def tanh(x): return 50 * np.tanh(x/20) + 50
def leaky_relu(x, alpha=0.01): return np.where(x > -65, x + 65, alpha * (x + 65))

relu_vals = relu(x)
sigmoid_vals = sigmoid(x)
tanh_vals = tanh(x)
leaky_relu_vals = leaky_relu(x)

### Visual Comparison

Compare biological and artificial neuron responses.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
fig.suptitle("Biological vs Artificial Neurons", fontsize=16, fontweight='bold', y=1.02)

# Left: Biological Neuron (Hodgkin–Huxley)
ax1 = axes[0]
ax1.plot(time_hh, hh_voltage, color='royalblue', linewidth=2.5, label='HH Membrane Potential')
ax1.axhline(-65, color='gray', linestyle='--', alpha=0.6, label='Resting potential (-65 mV)')
ax1.axhline(0, color='red', linestyle='--', alpha=0.4, label='Spike threshold (0 mV)')
ax1.axvspan(70, 130, alpha=0.1, color='gold', label='Stimulus period')

ax1.set_title("Biological Neuron (Hodgkin–Huxley Model)", fontsize=13, fontweight='bold')
ax1.set_xlabel("Time (ms)", fontsize=11)
ax1.set_ylabel("Membrane Potential (mV)", fontsize=11)
ax1.set_xlim(0, 200)
ax1.set_ylim(-70, 120)
ax1.grid(True, alpha=0.3)
ax1.legend(fontsize=9, loc='upper right')

ax1.annotate('Action Potential', xy=(95, 35), xytext=(135, 30),
             arrowprops=dict(arrowstyle='->', lw=2, color='red'),
             fontsize=10, color='red', fontweight='bold')


# Right: Artificial Neuron Activation Functions
ax2 = axes[1]
ax2.plot(x, 100 * (1 / (1 + np.exp(-x/10))), '--', color='orange', linewidth=2.5, label='Sigmoid')
ax2.plot(x, 50 * np.tanh(x/20) + 50, color='green', linewidth=2.5, label='Tanh')
ax2.plot(x, np.maximum(0, x + 65), color='purple', linewidth=2.5, label='ReLU')
ax2.plot(x, np.where(x > -65, x + 65, 0.01 * (x + 65)), '--', color='magenta', linewidth=2.0, label='Leaky ReLU')

# Add reference lines
ax2.axvline(-65, color='gray', linestyle='--', alpha=0.6)
ax2.axvline(0, color='red', linestyle='--', alpha=0.4)
ax2.text(-72, 105, 'Resting', color='gray', fontsize=9)
ax2.text(3, 105, 'Threshold', color='red', fontsize=9)

ax2.set_title("Artificial Neurons (Activation Functions)", fontsize=13, fontweight='bold')
ax2.set_xlabel("Input (a.u.)", fontsize=11)
ax2.set_ylabel("Output (a.u.)", fontsize=11)
ax2.set_xlim(-80, 40)
ax2.set_ylim(-10, 110)
ax2.grid(True, alpha=0.3)
ax2.legend(fontsize=9, loc='upper left')

plt.tight_layout()
plt.show()



SyntaxError: invalid syntax. Perhaps you forgot a comma? (1869904564.py, line 26)

### Quantitative Comparison

Compare biological (HH) and artificial neurons on key computational properties.

In [5]:
print("="*70)
print("QUANTITATIVE COMPARISON: BIOLOGICAL vs ARTIFICIAL")
print("="*70)

# Spike duration
spike_indices = np.where(hh_voltage > 0)[0]
if len(spike_indices) > 0:
    spike_duration = (spike_indices[-1] - spike_indices[0]) * 0.01
    print(f"Spike duration: {spike_duration:.2f} ms")

# Spike count
spike_count = np.sum((hh_voltage[:-1] < 0) & (hh_voltage[1:] > 0))
print(f"Spike count: {spike_count}")
print(f"Peak voltage: {np.max(hh_voltage):.2f} mV")
print(f"Range: {np.max(hh_voltage)-np.min(hh_voltage):.1f} mV")

print("\nArtificial neurons:")
print(" - Instantaneous, continuous outputs")
print(" - ReLU: piecewise linear; Sigmoid/Tanh: smooth & bounded")

QUANTITATIVE COMPARISON: BIOLOGICAL vs ARTIFICIAL
Spike duration: 138.76 ms
Spike count: 5
Peak voltage: 104.32 mV
Range: 115.0 mV

Artificial neurons:
 - Instantaneous, continuous outputs
 - ReLU: piecewise linear; Sigmoid/Tanh: smooth & bounded


### Insights: Bridging biological and artificial intelligence.

In [6]:
insights = [
    ("Spiking Neural Networks", "Time-dependent spikes encode info", 
     "Standard ANNs are timeless", 
     "Add temporal dynamics → event-driven computation"),
    ("Energy Efficiency", "Sparse spiking activity", 
     "Dense computation (all neurons active)", 
     "Event-driven computation saves energy"),
    ("Parameter Inference", "Estimate HH parameters", 
     "Learn ANN weights", 
     "Both optimize parameters for target behavior"),
]

for title, bio, art, idea in insights:
    print(f"\n{title}\n" + "─"*60)
    print(f"  Biological: {bio}")
    print(f"  Artificial: {art}")
    print(f"  NeuroAI Idea: {idea}")


Spiking Neural Networks
────────────────────────────────────────────────────────────
  Biological: Time-dependent spikes encode info
  Artificial: Standard ANNs are timeless
  NeuroAI Idea: Add temporal dynamics → event-driven computation

Energy Efficiency
────────────────────────────────────────────────────────────
  Biological: Sparse spiking activity
  Artificial: Dense computation (all neurons active)
  NeuroAI Idea: Event-driven computation saves energy

Parameter Inference
────────────────────────────────────────────────────────────
  Biological: Estimate HH parameters
  Artificial: Learn ANN weights
  NeuroAI Idea: Both optimize parameters for target behavior


## Summary

- **Biological neurons** are dynamic, sparse, and energy-efficient  
- **Artificial neurons** are simple, fast, and parallel  
- **NeuroAI** combines both approaches for interpretable, efficient intelligence  
- **Your parameter inference project** connects Bayesian neuroscience with machine learning